# Chapter 3
> 1
>> **常用API位置表格**  
>> 1.1 非模型状态下Conv2D，要记得做权重初始化  
>> 1.2 BN使用示例   

> 2  
>> 

- 早期通过提取一张图片的    (颜色，形状，纹理)    这三个要素提取图像特征

- 现代计算机视觉经典四个任务:
![title](https://img2020.cnblogs.com/blog/1303526/202003/1303526-20200323105021295-550424085.png)

- Yang Lecun 引入了CNN解决视觉任务,多层卷积和池化层组合作用在输入图片上,在网络最后通常会加入一系列全连接层,ReLU激活函数一般加在卷积或者全连接的输出上,网络中通常还会加入Dropout来防止过拟合.

- 卷积层的实现方式其实是数学中的"互相关运算(Cross-correlation)"

- 为了避免卷积之后,图片尺寸变小,会进行填充(padding)操作

- $$H_{out} = \frac{ H_{in}+2P-K}{S}$$

- 通常将卷积核的输出通道数叫做卷积核的个数:
![title](https://gimg2.baidu.com/image_search/src=http%3A%2F%2Fimage1.bubuko.com%2Finfo%2F202105%2F20210512203117547717.png&refer=http%3A%2F%2Fimage1.bubuko.com&app=2002&size=f9999,10000&q=a80&n=0&g=0n&fmt=jpeg?sec=1644309097&t=0bebd078f2bb99e98624704c00570ff5)

- paddle.fluid.dygraph.Conv2D(num_channels,num_filters,filter_size,stride,padding,act)
- Conv2D输入是(N,C,H,W)大小

|常用函数|导入方式/作用|
|--|--|
|fluid|import paddle.fluid as fluid|
|Linear|from paddle.fluid.dygraph import Linear|
|Conv2D|from paddle.fluid.dygraph.nn import Conv2D|
|Pool2D|from paddle.fluid.dygraph.nn import Pool2D|
|fluid.dygraph.Layer|模型要继承这个类|
|SGD|fluid.optimizer.SGD|
|dygraph.to_variable|numpy转为tensor|
|tensor_x.numpy()|tensor转numpy|
|np.random.shuffle|打乱numpy|
|fluid.layers.square_error_cost|平方差损失函数|
|fluid.layers.mean|求完总损失记得求平均|
|paddle.dataset.mnist,train()|mnist数据集|
|paddle.batch|SGD直接分好批次|
|from visualdl import LogWriter|可视化训练过程|
|fluid.layers.accuracy|类似计算损失,可以计算准确率的API|
|fluid.save_dygraph|存储模型参数/优化器参数|
|BatchNorm|from paddle.fluid.dygraph.nn import BatchNorm|
|fluid.layers.dropout|有两种dropout方法|
|paddle.summary(lenet, (64, 1, 28, 28))|查看模型各层信息|





In [ ]:
# 不在模型中,而是指定卷积核的内容对图像进行卷积操作.这也是传统视觉算法中的方法
# 1,使用特定的卷积核进行图像边缘检测
import matplotlib.pyplot as plt 
from PIL import Image
import numpy as np 
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import Conv2D
from paddle.fluid.initializer import NumpyArrayInitializer

img = Image.open('./spong.jpg')
with fluid.dygraph.guard():
    w = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]],dtype='float32')/8
    w = w.reshape([1,1,3,3])

    w = np.repeat(w,3,axis=1)
    conv =  Conv2D(num_channels=3,num_filters=1,filter_size=[3,3],param_attr=fluid.ParamAttr(initializer=NumpyArrayInitializer(value=w)))
    #进行Conv操作的图片必须是(N,C,H,W)
    x = np.array(img).astype('float32') #这时图片是[H,W,3]
    x = np.transpose(x,(2,0,1)) #transpose的作用就是调换行列索引值
    x = x.reshape(1,3,img.height,img.width)
    x = fluid.dygraph.to_variable(x)
    y = conv(x)
    out = y.numpy()

plt.figure(figsize=(20,10))
f = plt.subplot(121)
f.set_title('input_image',fontsize=15)
plt.imshow(img)
f = plt.subplot(122)
f.set_title('output_image',fontsize=15)
plt.imshow(out.squeeze(),cmap='gray')
plt.show()

In [ ]:
# 2,使用BN示例
import numpy as np 
import paddle
import paddle.fluid as fluid
from paddle.fluid.dygraph.nn import BatchNorm 

data = np.array([[1,2,3],[4,5,6],[7,8,9]]).astype('float32')
with fluid.dygraph.guard():
    bn = BatchNorm(num_channels=3)
    x = fluid.dygraph.to_variable(data)
    y = bn(x)
    print(y.numpy())
'''
paddle里的BatchNorm是有记忆性的，最后测试时使用的均值和方差是最后一次训练时候Batch里面的均值和方差
'''    

In [ ]:
# 3,dropout例程
'''
在预测场景，会向前传递所有神经元的信号，可能会引起一个新的问题：训练时由于部分神经元被随机丢弃了，输出总数据的大小会变小。比如：计算L1时会比不使用Dropout小
Paddle提供了两种方法解决这一问题：
(1)downgrade_in_infer
训练时以比例r随即丢弃一部分神经元，不向后传递他的信号，预测时向后传递所有神经元的信号，但是每个神经元上的数值 * (1-r)
(2)upscale_in_train
训练时以比例r随机丢弃一部分神经元，不向后传递它们的信号；但是将那些被保留的神经元上的数值除以(1-r)；预测时向后传递所有神经元的信号，不做任何处理。
'''
import numpy as np
import paddle
import paddle.fluid as fluid
np.random.seed(2022)
# 创建(N,C,H,W)的数据,data1用来演示卷积层进行dropout之后的情况
data1 = np.random.rand(2,3,3,3).astype('float32')
# 创建(N,K)的数据，data2用来演示全连接层进行dropout之后的情况
data2 = np.arange(1,13).reshape([-1,3]).astype('float32')

with fluid.dygraph.guard():
    x1 = fluid.dygraph.to_variable(data1)
    out1_1 = fluid.dygraph.nn.Dropout(p=0.5,is_test=False)(x1)
    out1_2 = fluid.dygraph.nn.Dropout(p=0.5,is_test=True)(x1)

    x2 = fluid.dygraph.to_variable(data2)
    out2_1 = fluid.dygraph.nn.Dropout(p=0.5,is_test=False)(x2)
    out2_2 = fluid.dygraph.nn.Dropout(p=0.5,is_test=True)(x2)

    print('conv_out of train:{}\nconv_out of test:{}\nfc_out of train:{}\nfc_out of test:{}'.format(out1_1.numpy(),out1_2.numpy(),out2_1.numpy(),out2_2.numpy()))


## 2,多种经典网络图像分类(眼疾分类数据集  iChallenge-PM)
|model name|upload date|
|--|--|
|LeNet|1998,LeCun|
|AlexNet|2012,Alex|
|VGG|2014,Symonyan|
|GoogleNet|2014,Christian Szegedy|
|ResNet|2015,KaiMing He|

#### 课本上使用了对于以上模型全部手动实现，这里我们不这么干，太慢了。Paddle.vision.models内置了很多模型，使用paddle.vision.models.__all__可以查看都有哪些模型可以使用

![title](http://tiebapic.baidu.com/forum/w%3D580/sign=8de3193027d9f2d3201124e799ed8a53/cdcdab0f4bfbfbed5d7310ae3df0f736aec31f9f.jpg)

In [ ]:
# !unzip -q data/data23828/training.zip
# !unzip -q data/data23828/valid_gt.zip
# !unzip -q data/data23828/validation.zip
#!unzip -o -q PALM-Training400/PALM-Training400.zip
#-o的意思是不必询问用户，-o后直接覆盖原有文件

##### 认识数据集
- /data/data19065/下面包含(training.zip,validation.zip,valid_gt.zip)分别是(训练集图片及标签,验证集图片,验证集标签)
- 上面三个文件解压之后分别是(PAL-Training400，PALM-Validation，PALM-Validation-GT)
- 解压PALM-Training400.zip文件之后我们发现里面有三种类型的图片
> - P(病理性近视，正样本) 
> - N(正常眼睛，负样本)
> - H(高度近视，负样本)

- 我们选取正样本标记为1，负样本标记为0

In [ ]:
import os
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
from PIL import Image 
import cv2
import random

# Step1 数据预处理、建立data_loader
DATADIR = "PALM-Training400"
print('shape of picture:{}'.format(  np.array(Image.open(os.path.join(DATADIR,'H0001.jpg'))).shape  )) #都是(1444,1444,3)

# Step 1.1 把每张图缩放至(224,224,3)并且像素值归一化到[-1,1] 之间
def transform_img(img):
    img = cv2.resize(img,(224,224))
    #将[h,w,c]变成[c,h,w]
    img = np.transpose(img,(2,0,1)).astype('float32')
    img = img/255
    img = 2*img - 1
    return img
# Step 1.2 定义data_loader
def data_loader():
    BATCH_SIZE = 10
    def data_generator():
        img_batch = []
        label_batch = []
        DATADIR = "PALM-Training400"
        filenames = os.listdir(DATADIR)
        random.shuffle(filenames)
        length = len(filenames) 
        for filename in filenames:
            img = transform_img(cv2.imread(os.path.join(DATADIR,filename)))
            if filename[0] == 'P':
                label = 1
            else:
                label = 0
            img_batch.append(img)
            label_batch.append(label)
            if len(img_batch) == BATCH_SIZE:
                yield np.array(img_batch).astype('float32'),np.array(label_batch).astype('float32').reshape(-1,1)
                img_batch = []
                label_batch = []

        if len(img_batch) > 1:
                print('lalala')
                yield np.array(img_batch).astype('float32'),np.array(label_batch).astype('float32').reshape(-1,1)

    return data_generator


In [ ]:
import paddle
import paddle.fluid as fluid
import numpy as np 
from paddle.fluid.dygraph.nn import Conv2D,Pool2D,Linear,BatchNorm
from paddle.fluid.dygraph.base import to_variable
from paddle.fluid.layer_helper import LayerHelper

print(paddle.vision.models.__all__)
use_model = 'GoogleNet'#在'LeNet','AlexNet','VGG','GoogleNet','ResNet'

#很好paddl.vision.models里面没有Alexnet和GoogleNet
# Step2 手动实现Alexnet和GoogleNet
# Step2.1 AlexNet
if use_model == 'AlexNet':
    class AlexNet(fluid.dygraph.Layer):
        def __init__(self,num_classes):
            super(AlexNet,self).__init__()
            self.conv1 = Conv2D(num_channels=3,num_filters=96,filter_size=11,stride=4,padding=5,act='relu')
            self.pool1 = Pool2D(pool_size=2,pool_stride=2,pool_type='max')
            self.conv2 = Conv2D(num_channels=96,num_filters=256,filter_size=5,stride=1,padding=2,act='relu')
            self.pool2 = Pool2D(pool_size=2,pool_stride=2,pool_type='max')
            self.conv3 = Conv2D(num_channels=256,num_filters=384,filter_size=3,stride=1,padding=1,act='relu')
            self.conv4 = Conv2D(num_channels=384,num_filters=384,filter_size=3,stride=1,padding=1,act='relu')
            self.conv5 = Conv2D(num_channels=384,num_filters=256,filter_size=3,stride=1,padding=1,act='relu')
            self.pool5 = Pool2D(pool_size=2,pool_stride=2,pool_type='max')
            
            self.fc1 = Linear(input_dim=256*49,output_dim=64*49,act='relu')
            self.drop_ratio1 = 0.5
            self.fc2 = Linear(input_dim=64*49,output_dim=64*49,act='relu')
            self.drop_ratio2 = 0.5
            self.fc3 = Linear(input_dim=64*49,output_dim=num_classes)
   
        def forward(self,x,label):
            x = self.conv1(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = self.pool2(x)
            x = self.conv3(x)
            x = self.conv4(x)
            x = self.conv5(x)
            x = self.pool5(x)
            x = fluid.layers.reshape(x,[x.shape[0],-1])
            x = self.fc1(x)
            x = fluid.layers.dropout(x,self.drop_ratio1)
            x = self.fc2(x)
            x = fluid.layers.dropout(x,self.drop_ratio2)
            x = self.fc3(x)
            acc = fluid.layers.accuracy(input=x,label=label)
            return x,acc

if use_model == 'GoogleNet':
    # Step 2.2.1 定义Googlenet的Inception块
    class Inception(fluid.dygraph.Layer):
        def __init__(self,in_channels,c1,c2,c3,c4):
            super(Inception,self).__init__()
            #依次创建Inception每条支路上使用到的操作
            self.p1_1 = Conv2D(num_channels=in_channels,num_filters=c1,filter_size=1,act='relu')
            self.p2_1 = Conv2D(num_channels=in_channels,num_filters=c2[0],filter_size=1,act='relu')
            self.p2_2 = Conv2D(num_channels=c2[0],num_filters=c2[1],filter_size=3,padding=1,act='relu')
            self.p3_1 = Conv2D(num_channels=in_channels,num_filters=c3[0],filter_size=1,act='relu')
            self.p3_2 = Conv2D(num_channels=c3[0],num_filters=c3[1],filter_size=5,padding=2,act='relu')
            self.p4_1 = Pool2D(pool_size=3,pool_stride=1,pool_padding=1)
            self.p4_2 = Conv2D(num_channels=in_channels,num_filters=c4,filter_size=1,act='relu')
        def forward(self,x):
            p1 = self.p1_1(x)
            p2 = self.p2_2(self.p2_1(x))
            p3 = self.p3_2(self.p3_1(x))
            p4 = self.p4_2(self.p4_1(x))
            return fluid.layers.concat([p1,p2,p3,p4],axis=1)
            
    # Step 2.2.2 定义GoogleNet
    class GoogleNet(fluid.dygraph.Layer):
            def __init__(self,num_classes):
                super(GoogleNet,self).__init__()
                self.conv1 = Conv2D(num_channels=3,num_filters=64,filter_size=7,padding=3,act='relu')
                self.pool1 = Pool2D(pool_size=3,pool_stride=2,pool_padding=1,pool_type='max')
                self.conv2_1 = Conv2D(num_channels=64,num_filters=64,filter_size=1,act='relu')
                self.conv2_2 = Conv2D(num_channels=64,num_filters=192,filter_size=3,padding=1,act='relu')
                self.pool2 = Pool2D(pool_size=3,pool_stride=2,pool_padding=1,pool_type='max')

                self.block3_1 = Inception(192,64,(96,128),(16,32),32)
                self.block3_2 = Inception(256,128,(128,192),(32,96),64)

                self.pool3 = Pool2D(pool_size=3,pool_stride=2,pool_padding=1,pool_type='max')

                self.block4_1 = Inception(480,192,(96,208),(16,48),64)
                self.block4_2 = Inception(512,160,(112,224),(24,64),64)
                self.block4_3 = Inception(512,128,(128,256),(24,64),64)
                self.block4_4 = Inception(512,112,(144,288),(32,64),64)
                self.block4_5 = Inception(528,256,(160,320),(32,128),128)

                self.pool4 = Pool2D(pool_size=3,pool_stride=2,pool_padding=1,pool_type='max')

                self.block5_1 = Inception(832,256,(160,320),(32,128),128)
                self.block5_2 = Inception(832,384,(192,384),(48,128),128)

                self.pool5 = Pool2D(pool_stride=1,global_pooling=True,pool_type='avg')
                self.fc = Linear(input_dim=1024,output_dim=num_classes,act=None)

            def forward(self,x):
                x = self.pool1(self.conv1(x))
                x = self.pool2(self.conv2_2(self.conv2_1(x)))
                x = self.pool3(self.block3_2(self.block3_1(x)))
                x = self.block4_3(self.block4_2(self.block4_1(x)))
                x = self.pool4(self.block4_5(self.block4_4(x)))
                x = self.pool5(self.block5_2(self.block5_1(x)))
                x = fluid.layers.reshape(x,[x.shape[0],-1])
                x = self.fc(x)
                return x
'''
# Step3 模型选择
if use_model == 'LeNet':
    model = paddle.vision.models.LeNet(num_classes=1)
if use_model == 'AlexNet':
    model = AlexNet(num_classes=1)
if use_model == 'VGG':
    model = paddle.vision.models.VGG(num_classes=1)
if use_model == 'GoogleNet':
    model = GoogleNet(num_classes=1)
if use_model == 'ResNet':
    model = paddle.vision.models.ResNet(num_classes=1)

paddle.summary(model, (64, 3, 224, 224))
'''

In [ ]:
 # Step4 开始训练
 '''
 with fluid.dygraph.guard(place=fluid.CUDAPlace(0)):    
    # Step3 模型选择
    if use_model == 'LeNet':
        model = paddle.vision.models.LeNet(num_classes=1)
    if use_model == 'AlexNet':
        model = AlexNet(num_classes=1)
    if use_model == 'VGG':
        model = paddle.vision.models.VGG(num_classes=1)
    if use_model == 'GoogleNet':
        model = GoogleNet(num_classes=1)
    if use_model == 'ResNet':
        model = paddle.vision.models.ResNet(num_classes=1)

    #paddle.summary(model, (64, 3, 224, 224))

    optimizer = fluid.optimizer.Momentum(learning_rate=0.0001,momentum=0.5,parameter_list=model.parameters())
    EPOCH_NUM = 20
    min_loss = 0.05
    for epoch in range(EPOCH_NUM):
        train_reader = data_loader()
        try:
            for batch_id,data in enumerate(train_reader()):
                img_batch,label_batch = data
                img_batch_tensor = fluid.dygraph.to_variable(img_batch)
                label_batch_tensor = fluid.dygraph.to_variable(label_batch)
                predict = model(img_batch_tensor)
                loss = fluid.layers.square_error_cost(predict,label_batch_tensor)
                avg_loss = fluid.layers.mean(loss)
                if avg_loss.numpy() <= min_loss:
                    fluid.save_dygraph(model.state_dict(),'out/model')
                    fluid.save_dygraph(optimizer.state_dict(),'out/model')
                    min_loss = avg_loss.numpy()
                
                if batch_id % 10 == 0:
                    print('iter:{} of epoch:{},loss is {}'.format(batch_id,epoch,avg_loss.numpy()))
                avg_loss.backward()
                optimizer.minimize(avg_loss)
                model.clear_gradients()
        except:
            pass
    '''
    

In [ ]:
# 随便找俩图测试一波
img_1 = fluid.dygraph.to_variable(np.array([transform_img(cv2.imread('PALM-Training400/N0002.jpg'))]).astype('float32'))
img_2 = fluid.dygraph.to_variable(np.array([transform_img(cv2.imread('PALM-Training400/P0135.jpg'))]).astype('float32'))

params_dict,opt_dict = fluid.load_dygraph('out/model')
if use_model == 'LeNet':
    model = paddle.vision.models.LeNet(num_classes=1)
if use_model == 'AlexNet':
    model = AlexNet(num_classes=1)
if use_model == 'VGG':
    model = paddle.vision.models.VGG(num_classes=1)
if use_model == 'GoogleNet':
    model = GoogleNet(num_classes=1)
if use_model == 'ResNet':
    model = paddle.vision.models.ResNet(num_classes=1)

model.load_dict(params_dict)

predict = model(img_2).numpy()
predict


在这一节里，介绍了几种典型的图像分类模型，并将它们应用到眼疾筛查数据集上，除了LeNet不适用于大尺寸图象分类问题以外，其他几个模型在此问题上的准确率都在95%以上。